In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import project specific functions
from column_map import column_map
from yesno_functions import *
from format_datetime import *

In [3]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [4]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [5]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [6]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('ambassador_dpr')

In [7]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [8]:
cols = list(col_rename.values())

### Read the site reference list from SQL

In [9]:
sql = 'select * from crowdsdb.dbo.tbl_ref_sites'

In [10]:
site_ref = pd.read_sql(con = engine, sql = sql)[['site_id', 'site_desc', 'borough']]

### Read the current data from SQL

In [11]:
sql = 'select * from crowdsdb.dbo.tbl_dpr_ambassador'

In [19]:
ambass_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['ambassador_id', 'patroncount'])
              .fillna(value = np.nan, axis = 1))

In [20]:
sql_cols = list(ambass_sql.columns.values)

In [21]:
ambass_sql['encounter_timestamp'] = format_datetime(ambass_sql, 'encounter_timestamp')
ambass_sql['encounter_datetime'] = format_datetime(ambass_sql, 'encounter_datetime')

In [22]:
hash_rows(ambass_sql, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Read the latest data from Google Sheets

In [23]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
client = gspread.authorize(creds)

In [24]:
sheet = client.open('_Combined INTERNAL PARKS Ambassador COVID-19 Reporting (Responses)')

In [25]:
ws = sheet.worksheet('Form Responses 1')

In [26]:
ambass = (get_as_dataframe(ws, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [27]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
1,6/1/2020 16:56:09,6/1/2020 16:20:00,Juniper Valley Park-Juniper South Playground,NaN,Recreation,Eric,Friedman,NaN,NaN,NaN,...,Bench/sitting area,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
2,6/1/2020 16:53:11,6/1/2020 15:40:00,Juniper Valley Park-Juniper North Playground T...,NaN,Recreation,Eric,Friedman,NaN,NaN,NaN,...,Open field/multi-purpose play area,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
3,6/1/2020 16:44:57,6/1/2020 15:30:00,Detective Keith L Williams Park,NaN,Recreation,Jordan,Calloway,NaN,NaN,NaN,...,NaN,NaN,NaN,Playground,2.0,Yes,Yes,No,NaN,Queens
4,6/1/2020 16:43:57,6/1/2020 14:00:00,Rochdale Park,NaN,Recreation,Jordan,Calloway,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
5,6/1/2020 16:43:09,6/1/2020 10:00:00,Detective Keith L Williams Park,NaN,Recreation,Jordan,Calloway,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens


In [28]:
yesno = ['sd_pdcontact', 'closed_approach', 'closed_outcome', 'closed_pdcontact']

In [29]:
yesno_cols(ambass, yesno)

In [30]:
#Remove rows with no timestamp because these rows have no data
ambass = ambass[ambass['encounter_timestamp'].notnull()]

In [31]:
ambass = ambass.merge(site_ref, how = 'left', on = ['site_desc', 'borough'])#[sql_cols]

In [33]:
ambass_sql['encounter_timestamp'] = format_datetime(ambass_sql, 'encounter_timestamp')
ambass_sql['encounter_datetime'] = format_datetime(ambass_sql, 'encounter_datetime')

In [34]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_comments,closed_amenity,closed_patroncount,closed_comments,borough,sd_pdcontact,closed_approach,closed_outcome,closed_pdcontact,site_id
0,6/1/2020 16:56:09,6/1/2020 16:20:00,Juniper Valley Park-Juniper South Playground,NaN,Recreation,Eric,Friedman,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Queens,0.0,NaN,NaN,NaN,Q102-03
1,6/1/2020 16:53:11,6/1/2020 15:40:00,Juniper Valley Park-Juniper North Playground T...,NaN,Recreation,Eric,Friedman,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Queens,0.0,NaN,NaN,NaN,Q102-02
2,6/1/2020 16:44:57,6/1/2020 15:30:00,Detective Keith L Williams Park,NaN,Recreation,Jordan,Calloway,NaN,NaN,NaN,...,NaN,Playground,2.0,NaN,Queens,NaN,1.0,1.0,0.0,Q121
3,6/1/2020 16:43:57,6/1/2020 14:00:00,Rochdale Park,NaN,Recreation,Jordan,Calloway,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Queens,NaN,NaN,NaN,NaN,Q413
4,6/1/2020 16:43:09,6/1/2020 10:00:00,Detective Keith L Williams Park,NaN,Recreation,Jordan,Calloway,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Queens,NaN,NaN,NaN,NaN,Q121


In [26]:
hash_rows(ambass, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Find the deltas based on the row hashes

In [27]:
ambass_deltas = (check_deltas(new_df = ambass, old_df = ambass_sql, on = ['encounter_timestamp', 'site_id'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [28]:
ambass_deltas.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough,dml_verb
0,6/1/2020 12:35:23,5/31/2020 15:30:00,M010,NaN,Recreation,Jasmine,Campbell,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,I
1,6/1/2020 12:32:55,6/1/2020 12:00:00,Q395,NaN,Recreation,Dion,Battey,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens,I
2,6/1/2020 12:32:10,6/1/2020 10:00:00,Q218,NaN,Recreation,Dion,Battey,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens,I
3,6/1/2020 12:20:18,6/1/2020 12:15:00,Q014,NaN,Recreation,Karen,Dean,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens,I
4,6/1/2020 12:03:14,6/1/2020 11:00:00,M014,NaN,Recreation,Ahmad,Sarwary,Cecil,Jones,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,I


In [29]:
ambass_inserts = ambass_deltas[ambass_deltas['dml_verb'] == 'I'][sql_cols]

In [30]:
ambass_inserts.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
0,6/1/2020 12:35:23,5/31/2020 15:30:00,M010,NaN,Recreation,Jasmine,Campbell,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
1,6/1/2020 12:32:55,6/1/2020 12:00:00,Q395,NaN,Recreation,Dion,Battey,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
2,6/1/2020 12:32:10,6/1/2020 10:00:00,Q218,NaN,Recreation,Dion,Battey,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
3,6/1/2020 12:20:18,6/1/2020 12:15:00,Q014,NaN,Recreation,Karen,Dean,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
4,6/1/2020 12:03:14,6/1/2020 11:00:00,M014,NaN,Recreation,Ahmad,Sarwary,Cecil,Jones,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan


In [31]:
ambass_inserts.to_sql('tbl_dpr_ambassador', engine, index = False, if_exists = 'append')

In [32]:
ambass_updates = ambass_deltas[ambass_deltas['dml_verb'] == 'U'][sql_cols]

In [33]:
ambass_updates.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough


In [34]:
sql_update(ambass_updates, 'tbl_dpr_ambassador', engine, ['encounter_timestamp', 'site_id'])